In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
! pip install -q farm-haystack[colab,preprocessing,elasticsearch,metrics,inference]

In [3]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [4]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [5]:
from haystack.utils import launch_es

launch_es()

In [6]:
import time

time.sleep(30)

In [7]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

doc_index = "tutorial5_docs"
label_index = "tutorial5_labels"

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

# Connect to Elasticsearch
document_store = ElasticsearchDocumentStore(
    host=host,
    username="",
    password="",
    index=doc_index,
    label_index=label_index,
    embedding_field="emb",
    embedding_dim=768,
    excluded_meta_data=["emb"],
)

In [19]:
from haystack.nodes import PreProcessor

# Add evaluation data to Elasticsearch Document Store
# We first delete the custom tutorial indices to not have duplicate elements
# and also split our documents into shorter passages using the PreProcessor
preprocessor = PreProcessor(
    split_by="word",
    language = 'fa',
    split_length=200,
    split_overlap=0,
    split_respect_sentence_boundary=False,
    clean_empty_lines=False,
    clean_whitespace=False,
)
document_store.delete_documents(index=doc_index)
document_store.delete_documents(index=label_index)

# The add_eval_data() method converts the given dataset in json format into Haystack document and label objects. Those objects are then indexed in their respective document and label index in the document store. The method can be used with any dataset in SQuAD format.

document_store.add_eval_data(
    filename="/content/drive/MyDrive/dev_SQuAD_format.json",
    doc_index=doc_index,
    label_index=label_index,
    preprocessor=preprocessor,
)

Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 865.70docs/s]


In [20]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [10]:
# from haystack.nodes import FARMReader

# reader = FARMReader("deepset/roberta-base-squad2", top_k=2, return_no_answer=True)

# from haystack.pipelines import ExtractiveQAPipeline

# pipeline = ExtractiveQAPipeline(reader=reader, retriever=retriever)

In [21]:
from haystack import Pipeline

pipeline = Pipeline()

pipeline.add_node(component=retriever, name='Retriever', inputs=['Query'])

In [22]:
from haystack.schema import EvaluationResult, MultiLabel

# We can load evaluation labels from the document store
# We are also opting to filter out no_answer samples
eval_labels = document_store.get_all_labels_aggregated(drop_negative_labels=True, drop_no_answers=True)

In [23]:
eval_result = pipeline.eval(labels=eval_labels, params={"Retriever": {"top_k": 1}})

In [24]:
# !apt install -q libgraphviz-dev graphviz
# !pip install -q pygraphviz

Reading package lists...
Building dependency tree...
Reading state information...
graphviz is already the newest version (2.42.2-6).
libgraphviz-dev is already the newest version (2.42.2-6).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [25]:
# pipeline.draw()

In [26]:
retriever_result = eval_result["Retriever"]
retriever_result.head()

,multilabel_id,query,filters,gold_answers,context,gold_contexts,gold_id_match,context_match,answer_match,gold_id_or_answer_match,...,rank,document_id,gold_document_ids,gold_documents_id_match,gold_contexts_similarity,gold_answers_match,type,node,eval_mode,index
0,ea37d99686555082d0188b39a0c22882,فرشتگان ذکر شده در قرآن چه کسانی هستند؟,b'null',"[روح, جبرئیل]",الر، این آیات کتاب، و قرآن مبین است. کافران چ...,[روزی که «روح» و «ملائکه» در یک صف می‌ایستند و...,0.0,0.0,0.0,0.0,...,1.0,152f43fdd08031f41587ed924ff1b791-0,"[466a6e848419cde2543f10e4c9c2fb7b-0, b851bb133...","[0.0, 0.0]","[45.4293241786755, 45.062836624775585]","[0.0, 0.0]",document,Retriever,integrated,0
1,58bef4f41aeb56130f91ca0024bbb3ba,کدام پیامبر از سوی فرعونیان تکذیب شد؟,b'null',[موسی],اگر تو را تکذیب کنند، پیش از آنها قوم نوح و ع...,[اگر تو را تکذیب کنند، پیش از آنها قوم نوح و ...,1.0,1.0,1.0,1.0,...,1.0,e585378a61acfe7df4037e812678f784-0,[e585378a61acfe7df4037e812678f784-0],[1.0],[100.0],[1.0],document,Retriever,integrated,0
2,54889a0b99cb72a5291c400f0875e277,اعمال چه کسانی برایشان زینت داده شده است؟,b'null',[اسرافکاران],کسانی که بعد از روشن شدن هدایت برای آنها، پشت ...,[هنگامی که به انسان زیان رسد، ما را در حالی ...,0.0,0.0,0.0,0.0,...,1.0,c0c0bd46a151a4eabb4d0313ae2a9307-0,[55fe664a20ebefc258fc2bf356c99fe5-0],[0.0],[44.4043321299639],[0.0],document,Retriever,integrated,0
3,49c673ab2d0bb51cbc2d6855260f43b1,قوم عاد به چه بلایی گرفتار شدند؟,b'null',[تندبادی است که عذاب دردناکی در آن است],روزی است که مسلّماً واقع می‌شود! چه روز واقع ...,[ برادر قوم عاد را یاد کن، آن زمان که قومش را ...,0.0,0.0,0.0,0.0,...,1.0,f20712914af2445d69b7334ed6a40f18-0,[2ad37a1b4a4778c92fcbbaf6ae656e7c-0],[0.0],[43.34763948497854],[0.0],document,Retriever,integrated,0
4,76cd8d7d5b87be7f229920aa561526b7,جامهای بهشتی لبریز از چه چیزی است؟,b'null',[شراب طهوری آمیخته با زنجبیل],خداوند آنان را از شرّ آن روز نگه می‌دارد و آن...,[ خداوند آنان را از شرّ آن روز نگه می‌دارد و آ...,1.0,1.0,1.0,1.0,...,1.0,d36afc412b9a53a62b408d810d6cee8a-0,[d36afc412b9a53a62b408d810d6cee8a-0],[1.0],[100.0],[1.0],document,Retriever,integrated,0


In [27]:
eval_result.save("../")

In [28]:
saved_eval_result = EvaluationResult.load("../")
metrics = saved_eval_result.calculate_metrics()
print(f'Retriever - Recall (single relevant document): {metrics["Retriever"]["recall_single_hit"]}')
print(f'Retriever - Recall (multiple relevant documents): {metrics["Retriever"]["recall_multi_hit"]}')
print(f'Retriever - Mean Reciprocal Rank: {metrics["Retriever"]["mrr"]}')
print(f'Retriever - Precision: {metrics["Retriever"]["precision"]}')
print(f'Retriever - Mean Average Precision: {metrics["Retriever"]["map"]}')

# print(f'Reader - F1-Score: {metrics["Reader"]["f1"]}')
# print(f'Reader - Exact Match: {metrics["Reader"]["exact_match"]}')


Retriever - Recall (single relevant document): 0.6133333333333333
Retriever - Recall (multiple relevant documents): 0.6066666666666667
Retriever - Mean Reciprocal Rank: 0.6133333333333333
Retriever - Precision: 0.6133333333333333
Retriever - Mean Average Precision: 0.6066666666666667


In [ ]:
pipeline.print_eval_report(saved_eval_result)

In [ ]:
# advanced_eval_result = pipeline.eval(
#     labels=eval_labels, params={"Retriever": {"top_k": 1}}, sas_model_name_or_path="cross-encoder/stsb-roberta-large"
# )

# metrics = advanced_eval_result.calculate_metrics()
# print(metrics["Reader"]["sas"])
